In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
print(os.getenv("PINECONE_API_KEY"))



f839e0d6-ac96-40f8-85e8-5f864e058ba4


/home/devopsbro/miniconda3/envs/rag/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [18]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

pc.delete_index("rag")

pc.create_index(
    name="rag",
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [19]:
import json
data = json.load(open('review.json'))
data['reviews']

[{'professor': 'Dr. Emily Parker',
  'subject': 'Molecular Biology',
  'star': 4,
  'review': 'Complex biological concepts made accessible and interesting.'},
 {'professor': 'Prof. John Smith',
  'subject': 'World History',
  'star': 3,
  'review': 'Covers a broad range of historical events, but lacks depth.'},
 {'professor': 'Dr. Maria Rodriguez',
  'subject': 'Quantum Physics',
  'star': 5,
  'review': 'Mind-bending theories explained with clarity and enthusiasm.'},
 {'professor': 'Prof. David Lee',
  'subject': 'Literature',
  'star': 4,
  'review': 'Insightful analysis of classic and contemporary works.'},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Psychology',
  'star': 5,
  'review': 'Fascinating exploration of human behavior and cognition.'},
 {'professor': 'Prof. Michael Brown',
  'subject': 'Computer Science',
  'star': 2,
  'review': 'Programming concepts not explained clearly enough.'},
 {'professor': 'Dr. Laura White',
  'subject': 'Economics',
  'star': 4,
  'review

In [20]:
import google.generativeai as genai
import os

processed_data = []
for review in data['reviews']:
    
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
    result = genai.embed_content(
        model="models/text-embedding-004",
        content=review['review'],
        task_type="retrieval_document",
        title="Embedding of single string")
    embeddings = result['embedding']
    
    processed_data.append({
        "values": embeddings,
        "id": review['professor'],
        "metadata":{
            'review': review['review'],
            'subject': review['subject'],
            'star': review['star'],
        }
    })

In [21]:
index = pc.Index('rag')
index.upsert(
    vectors = processed_data,
    namespace='ns1'
    )

{'upserted_count': 20}

In [22]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}